In [ ]:
import identify_face_etc as id


In [ ]:
image_path = "../data/images/greenspillexample.png"
class_indices = [2, 3]
bin_mask, seg_img = id.here_we_go(og_img=image_path, class_indices=class_indices)
# NOTE: bin_mask and seg_img are numpy arrays

In [ ]:
id.save_as(fname="bin_mask.jpg", img=bin_mask)
id.save_as(fname="seg_img.jpg", img=seg_img)